In [1]:
from general_tools.notebook import gpu_utils
GPU = 0
gpu_utils.setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean
from sklearn.manifold import TSNE

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf

from tf_lab.in_out.basics import read_saved_epochs
                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.nips.helper import center_pclouds_in_unit_sphere, \
average_per_class, pclouds_with_zero_mean_in_unit_sphere

from sklearn.svm import LinearSVC
from general_tools.simpletons import sort_dict_by_key, sort_dict_by_val

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_samples = 2048
put_in_usphere = True
model_net = '40'

In [5]:
# NIPS - CONV
# experiment_name = 'all_snc_rotated_conv_arch_2048pts_chamfer'
# top_data_dir = '/scr/optas/DATA/nips/nn_models/vanilla_ae/'


# no_bnorm - deep- not wide (new architecture)
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iclr/nn_models/all_snc'
experiment_name ='all_snc_rotated_deeper_mlp_arch_1_pc2048pts_chamfer'
# - it requires c approx 100

# bnorm on encoder - deep- not wide (old architecture)
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iclr/nn_models/all_snc/'
experiment_name ='all_snc_rotated_deeper_mlp_arch_1_pc_bnorm_on_encoder_old_code2048pts_chamfer'
# - >  seems to work with both big and small c

# no bnorm - deep- and wide (new architecture)
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iclr/nn_models/all_snc/'
experiment_name = 'all_snc_rotated_deeper_wider_mlp_arch_1_pc2048pts_chamfer'
# 2K epochs (200, 0.2, True, 'squared_hinge'), 0.7947793757649938),
# 1Kepochs [((80, 2, True, 'squared_hinge'), 0.79456640146878821),

 
# no bnorm  - deep- not wide (old architecture)
# top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iclr/nn_models/all_snc/'
# experiment_name = 'all_snc_rotated_deeper_mlp_arch_1_pc_no_bnorm_old_code2048pts_chamfer'
# > epoch 600smething was around 81.
# > epoch 850 (40, 2, True, 'squared_hinge'), 0.80518268053855557)



top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/iclr/nn_models/all_snc/'
experiment_name = 'all_snc_rotated_deeper_mlp_arch_1_pc_with_bnorm_old_arch2048pts_chamfer'

In [13]:
train_dir = osp.join(top_data_dir, experiment_name)
conf = Conf.load(osp.join(train_dir, 'configuration'))

if conf.train_dir != train_dir: # added to address moving training-folder to other location
    conf.train_dir = train_dir
    conf.save(osp.join(conf.train_dir, 'configuration'))

print conf
conf.n_output = conf.n_input
conf.allow_gpu_growth = True

              allow_gpu_growth: False
                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'b_norm': True, 'layer_sizes': [512, 1024, 6144]}
                       encoder: encoder_with_convs_and_symmetry_new
                  encoder_args: {'filter_sizes': [1, 1, 1, 1, 1], 'n_filters': [64, 128, 128, 256, 512], 'b_norm': True, 'strides': [1, 1, 1, 1, 1]}
               experiment_name: all_snc_rotated_deeper_mlp_arch_1_pc_with_bnorm_old_arch2048pts_chamfer
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
             

In [14]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
saved_epochs = read_saved_epochs(conf.train_dir)
last_epoch = saved_epochs[-1]
# last_epoch = 1000
ae.restore_model(conf.train_dir, last_epoch, verbose=True)

Model restored in epoch 630.


In [8]:
model_net_dir = '/orions4-zfs/projects/optas/DATA/Point_Clouds/Model_Net_' + model_net +  '/from_manifold/'
model_net_dir = osp.join(model_net_dir, str(n_pc_samples))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

In [9]:
verbose = True
n_threads = 20
put_in_usphere = True
pc, model_names, labels = load_point_clouds_from_filenames(train_pc_files, n_threads, pc_loader, verbose)
if put_in_usphere:
#     pc = center_pclouds_in_unit_sphere(pc)
    pc = pclouds_with_zero_mean_in_unit_sphere(pc)
    
train_data = PointCloudDataSet(pc, labels=labels)

pc, model_names, labels = load_point_clouds_from_filenames(test_pc_files, n_threads, pc_loader, verbose)

if put_in_usphere:
#     pc = center_pclouds_in_unit_sphere(pc)
    pc = pclouds_with_zero_mean_in_unit_sphere(pc)
    
test_data = PointCloudDataSet(pc, labels=labels)

9841 pclouds were loaded. They belong in 40 shape-classes.
2467 pclouds were loaded. They belong in 40 shape-classes.


In [15]:
train_feed, train_latent, train_classes = ae.embedding_at_tensor(train_data, conf, tensor_name='bottleneck')
cids = classes_to_integers(int(model_net), train_classes)[1]

test_feed, test_latent, test_classes = ae.embedding_at_tensor(test_data, conf, tensor_name='bottleneck')
cids_ = classes_to_integers(int(model_net), test_classes)[1]

In [16]:
# old_c_space = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# new_c_space = [10, 100, 300]

# c_space = old_c_space + new_c_space
intercept_scaling_space = [0.2, 0.5, 1, 2, 3, 4]
dual_space = [False, True]
loss_space = ['hinge', 'squared_hinge']


# intercept_scaling_space = [0.3]
# loss_space = ['squared_hinge']

# c_space = [0.08, 0.1,] 1, 2, 5, 10, 20, 30, 40, 50, 80, 100, 120, 200, 220, 300]
c_space = [0.01, 0.02, 0.05, 0.08, 0.1]
all_scores = dict()

for c in c_space:
    for intercept in intercept_scaling_space:
        for dual in dual_space:
            for loss in loss_space:
                try:
                    lsvc = LinearSVC(C=c, loss=loss, intercept_scaling=intercept, dual=dual)
                    lsvc.fit(train_latent, cids)
                    print c, intercept, dual, loss
                    score = average_per_class(lsvc, test_latent, cids_)                
                    print score
                    all_scores[(c, intercept, dual, loss)] = score
                except:
                    continue

0.01 0.2 False squared_hinge
0.807589351285
0.01 0.2 True hinge
0.759963586291
0.01 0.2 True squared_hinge
0.807589351285
0.01 0.5 False squared_hinge
0.807589351285
0.01 0.5 True hinge
0.762029375765
0.01 0.5 True squared_hinge
0.807589351285
0.01 1 False squared_hinge
0.807589351285
0.01 1 True hinge
0.761820073439
0.01 1 True squared_hinge
0.808839351285
0.01 2 False squared_hinge
0.807589351285
0.01 2 True hinge
0.761529375765
0.01 2 True squared_hinge
0.807589351285
0.01 3 False squared_hinge
0.807589351285
0.01 3 True hinge
0.761529375765
0.01 3 True squared_hinge
0.807589351285
0.01 4 False squared_hinge
0.807589351285
0.01 4 True hinge
0.761279375765
0.01 4 True squared_hinge
0.807589351285
0.02 0.2 False squared_hinge
0.810339351285
0.02 0.2 True hinge
0.791048653611
0.02 0.2 True squared_hinge
0.810839351285
0.02 0.5 False squared_hinge
0.811089351285
0.02 0.5 True hinge
0.788007955936
0.02 0.5 True squared_hinge
0.811339351285
0.02 1 False squared_hinge
0.810339351285
0.02 1

In [17]:
sort_dict_by_val(all_scores, reverse=True)

[((0.1, 0.2, False, 'squared_hinge'), 0.83172705018359849),
 ((0.1, 2, False, 'squared_hinge'), 0.83172705018359849),
 ((0.1, 4, False, 'squared_hinge'), 0.83151774785801713),
 ((0.1, 3, False, 'squared_hinge'), 0.83147705018359852),
 ((0.1, 1, False, 'squared_hinge'), 0.83147705018359852),
 ((0.1, 0.5, False, 'squared_hinge'), 0.83147705018359852),
 ((0.1, 0.2, True, 'squared_hinge'), 0.83072705018359849),
 ((0.08, 0.2, False, 'squared_hinge'), 0.8290642594859241),
 ((0.08, 2, False, 'squared_hinge'), 0.8290642594859241),
 ((0.08, 0.5, False, 'squared_hinge'), 0.8290642594859241),
 ((0.08, 3, False, 'squared_hinge'), 0.82906425948592388),
 ((0.08, 4, False, 'squared_hinge'), 0.82906425948592388),
 ((0.08, 1, False, 'squared_hinge'), 0.82881425948592413),
 ((0.1, 0.5, True, 'squared_hinge'), 0.8273956548347613),
 ((0.1, 3, True, 'squared_hinge'), 0.82730844553243588),
 ((0.1, 1, True, 'squared_hinge'), 0.82654283965728281),
 ((0.08, 1, True, 'squared_hinge'), 0.82440146878824971),
 ((0

In [ ]:
# New stuff for ICLR
# 1.  Chamfer
# encoder_args: {'filter_sizes': [1, 1, 1, 1, 1], 'strides': [1, 1, 1, 1, 1], 'n_filters': [64, 128, 128, 256, 512], 'dropout_prob': [0, 0, 0, 0, 0.5], 'b_norm': True}
# decoder_args: {'b_norm': True, 'layer_sizes': [512, 1024, 6144]}
# -> @ 980 epochs of training: best was on model-net40:
# 0.6 1 True hinge
# 0.840083537332
# -> @ 1620 epochs of training: best was on model-net40:
# 0.4 1 False squared_hinge
# 0.843783353733
# at @ 2000 epochs of training: best was on model-net40:
# (0.3, 0.2, False, 'squared_hinge'), 0.84087423500612002)


In [ ]:
# FROM NIPS
# USING ALL ShapeNetCore DATA : ~57K models.
# Training at (max) 2K epochs.

# EMD, 2K, model_net_40
# 1.0 3 False squared_hinge
# 0.844814259486
# 1.0 3 True squared_hinge
# 0.844814259486

# EMD, 2K, model_net_10
# 0.4 2 True hinge
# 0.953279069767


# Chamfer, 2K model_net_40
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.3 4 True squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.6 1 True squared_hinge
# 0.856264075887
# 0.7 1 False squared_hinge
# 0.856764075887

# Chamfer, 2K model_net_10
# 0.9 4 True hinge
# 0.950790697674
# 0.950790697674
# 0.8 0.5 False squared_hinge

# 1.0 4 True hinge
# 0.950790697674
# 1.0 3 True hinge
# 0.950790697674
# 0.9 4 True hinge
# 0.950790697674

# 0.7 4 True hinge
# 0.950790697674


# With WU 7 Classes:

# M10, CD, 2K epochs
# 0.4 4 True hinge
# 0.95176744186

# M40, CD, 2K epochs
# 1.0 1 True hinge
# 0.84410495716

# M10, EMD, ~2.5K epochs
# about 0.94

# M40, EMD, ~2.5K epochs
# about 0.83